## Lib

In [1]:
import pandas as pd
import numpy as np

from tslearn.neighbors import KNeighborsTimeSeries

from utils.evaluation import evaluate, get_binary_metrics
from utils.utils import read_dataset,split_dataset

C:\Users\delan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


## Transplant

In [2]:
%%time
x_test,y_test,x_val,y_val,y_val_true,x_train,y_train,_, col = np.load('data/transplant_1col.npy', allow_pickle=True)

x_test= np.concatenate([x_test, x_val])
y_test = np.concatenate([y_test, y_val])

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)

Wall time: 5.99 ms


In [3]:
np.mean(y_train)

0.42424242424242425

In [4]:
%%time
knn = KNeighborsTimeSeries(n_neighbors=5, metric="dtw")
knn.fit(x_train, y_train)
ind = knn.kneighbors(x_test, return_distance=False)
ypred = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), 1, y_train[ind])

Wall time: 5min 41s


In [5]:
evaluate(y_test,ypred)

,accuracy,recall,precision,f1-score
0,0.592593,0.578393,0.580611,0.578664


In [6]:
get_binary_metrics(y_test,ypred)

,recall,precision,f1-score
0,0.676471,0.635359,0.655271
1,0.480315,0.525862,0.502058


## ECG5000

In [16]:
%%time
x_test,y_test,x_val,y_val,x_train,y_train = np.load('data/ECG5000/ECG5000_full.npy', allow_pickle=True)

x_test = np.concatenate([x_test, x_val])
y_test = np.concatenate([y_test, y_val]) - 1
y_train = y_train -1

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)

Wall time: 24.9 ms


In [8]:
%%time
knn = KNeighborsTimeSeries(n_neighbors=1, metric="dtw")
knn.fit(x_train, y_train)
_, ind = knn.kneighbors(x_test)
ypred = y_test[ind]

Wall time: 35min 17s


In [9]:
evaluate(y_test,ypred)

,accuracy,recall,precision,f1-score
0,0.466222,0.199441,0.199462,0.199012


In [10]:
get_binary_metrics(y_test,ypred)

,recall,precision,f1-score
0,0.575942,0.590324,0.583044
1,0.363522,0.366286,0.364899
2,0.0348837,0.0198675,0.0253165
3,0.0228571,0.0208333,0.0217984
4,0,0,0


## MIT 

In [11]:
data,labels = read_dataset('../mitbih_train.csv','MIT-BIH')
x_train,y_train,x_val,y_val,y_val_true,x_test,y_test,y_test_true = split_dataset(data,labels)

x_train = np.concatenate([x_train, x_val])
y_train = np.concatenate([y_train, y_val])

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)

y_train_oh = y_train
y_test_oh = y_test

y_train = y_train.argmax(axis=1)
y_test = y_test.argmax(axis=1)

In [12]:
%%time
knn = KNeighborsTimeSeries(n_neighbors=3, metric="dtw")
knn.fit(x_train[:100,:], y_train[:100])
ind = knn.kneighbors(x_test[:4000,:], return_distance=False)
ypred = y_train[ind].max(axis=1)

Wall time: 6min 15s


In [13]:
evaluate(y_test[:4000], ypred)

,accuracy,recall,precision,f1-score
0,0.7955,0.496056,0.491064,0.445715


In [14]:
get_binary_metrics(y_test[:4000], ypred)

,recall,precision,f1-score
0,0.854223,0.932415,0.891608
1,0.205882,0.0871369,0.122449
2,0.242308,0.649485,0.352941
3,0.307692,0.0278746,0.0511182
4,0.870175,0.75841,0.810458
